In [2]:
import numpy as np
import cv2 as cv
#from matplotlib import pyplot as plt
from os import listdir
from os.path import isfile, join
from PIL import Image as Image
import PIL
import os
import math
import sys  

In [3]:
rgb_main_color_RGB = (173, 0, 30)
rgb_main_color_CMYK = {21, 100, 100, 15}


R_range = range(44, 123)
G_range = range(87, 151)
B_range = range(122, 175)

black = (0,0,0)
white = (255,255,255)

old_parts_folder = "old"
new_parts_folder = "new"
parts_types = [1,2,3,4,5,6,7,8,9,10,11,12]

In [4]:
def get_images_for_old(id):
    if id not in parts_types:
        raise
    out = []

    path = os.getcwd() + '\\' + old_parts_folder + '\\' + str(id) + '\\'
    files = [f for f in listdir(path) if isfile(join(path, f))]
    for file in files:
        im = cv.imread(path + file,0)
        out.append(im)
    return  out

In [5]:
def get_image_for_new(id):
    if id not in parts_types:
        raise

    path = os.getcwd() + '\\' + new_parts_folder + '\\' + str(id) + '\\'
    files = [f for f in listdir(path) if isfile(join(path, f))]
    if len(files) == 0:
        raise
    return cv.imread(path+files[0],0)

In [6]:
'''%matplotlib inline
def show(im, w = 40, h = 20):
    plt.figure(figsize = (w,h))
    
    if isinstance(im, str):
        plt.imshow(cv.imread(im,0)[::1])
    else:
        plt.imshow(np.array(im)[::1])'''

'%matplotlib inline\ndef show(im, w = 40, h = 20):\n    plt.figure(figsize = (w,h))\n    \n    if isinstance(im, str):\n        plt.imshow(cv.imread(im,0)[::1])\n    else:\n        plt.imshow(np.array(im)[::1])'

In [7]:
def RGB(im):    
    copy = im.copy()
    return copy.convert('RGB')

In [8]:
def remove_transparency(im, bg_colour=(255, 255, 255)):
    if im.mode in ('RGBA', 'LA') or (im.mode == 'P' and 'transparency' in im.info):
        alpha = im.convert('RGBA').split()[-1]
        bg = Image.new("RGBA", im.size, bg_colour + (255,))
        bg.paste(im, mask=alpha)
        return bg
    else:
        return im

In [9]:
def black_to_white(im):
    copy = im.copy()
    for i in range(copy.width):  # for each column
        for j in range(copy.height):  # For each row
            pixel = copy.getpixel((i, j))    
            if pixel[0] == 0 and pixel[1] == 0 and pixel[2] == 0:
                #print('black:',copy.getpixel((i, j)))
                copy.putpixel((i, j), white)
    return copy

In [10]:
def BW(im):
    copy = im.copy()
    for i in range(copy.width):  # for each column
        for j in range(copy.height):  # For each row
            print(copy.getpixel((i, j)))
            pixel = copy.getpixel((i, j))
            if (pixel[0] in R_range) and (pixel[1] in G_range) and (pixel[2] in B_range):
                copy.putpixel((i, j), white)
            else:
                copy.putpixel((i, j), black)
    
    return copy

In [11]:
def intersects(first, second):
    first_top_left = first[0]
    first_bottom_right = first[1]
    
    second_top_left = second[0]
    second_bottom_right = second[1]
    
    ftl_x = first_top_left[0]
    ftl_y = first_top_left[1]  
    fbr_x = first_bottom_right[0]
    fbr_y = first_bottom_right[1]
    
    
    stl_x = second_top_left[0]
    stl_y = second_top_left[1]  
    sbr_x = second_bottom_right[0]
    sbr_y = second_bottom_right[1]
    
    if len(set(range(ftl_x, fbr_x+1)) & set(range(stl_x, sbr_x+1))) > 0 and len(set(range(ftl_y, fbr_y+1)) & set(range(stl_y, sbr_y+1))) > 0:
        return True
    else:
        return False

In [40]:
def rgb_to_hsl(rgb):    
    r = float(rgb[0])
    g = float(rgb[1])
    b = float(rgb[2])
    high = max(r, g, b)
    low = min(r, g, b)
    h, s, v = ((high + low) / 2,)*3
    
    if high == low:
        h = 0.0
        s = 0.0
    else:
        d = high - low
        print(r,g,b)
        try:        
            s = d / (2 - high - low) if (high + low) / 2 > 0.5 else d / (high + low)
        except:
            s = 0
        h = {
            r: (g - b) / d + (6 if g < b else 0),
            g: (b - r) / d + 2,
            b: (r - g) / d + 4,
        }[high]
        h /= 6

    return (h, s, v)

def hsl_to_rgb(hsl):
    h = hsl[0]
    s = hsl[1]
    l = hsl[2]
    def hue_to_rgb(p, q, t):
        t += 1 if t < 0 else 0
        t -= 1 if t > 1 else 0
        if t < 1/6: return p + (q - p) * 6 * t
        if t < 1/2: return q
        if t < 2/3: p + (q - p) * (2/3 - t) * 6
        return p

    if s == 0:
        r, g, b = l, l, l
    else:
        q = l * (1 + s) if l < 0.5 else l + s - l * s
        p = 2 * l - q
        r = hue_to_rgb(p, q, h + 1/3)
        g = hue_to_rgb(p, q, h)
        b = hue_to_rgb(p, q, h - 1/3)

    return (int(r), int(g), int(b))

In [18]:
def replace_old_parts(template, trashhold=0.6, trash_pers = 0.5):
    method = eval('cv.TM_CCOEFF_NORMED')
    template_w, template_h = template.shape[::-1]
    temp = template.copy()
    
    
    parts_found = [] #((x_0,y_0) лево верх, (x_1, y_1) право низ)

    #по порядку проходимся по всем элементам
    for part_id in parts_types: 
        headers = ['итем ' + str(part_id), 'трешхолд']
        items = []
        #print('\nsearching for item', part_id)
        possible_old_items = get_images_for_old(part_id) # подгружаем все возможные варианты
        thresholds = [] #трешхолды для всех возможных вариантов
        top_left = []
        bottom_right = []
        
        for i, item in enumerate(possible_old_items):            
            res = cv.matchTemplate(item, temp, method) #threshlods  
            thresholds.append(np.max(res))
            
            min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)
            items.append((i, max_val))            
            top_left.append(max_loc)
            
            bottom_right.append((top_left[-1][0] + template_w, top_left[-1][1] + template_h))           
            
            #print(top_left[-1], bottom_right[-1])
        # нашли все трешхолды текущей части. 
        # Выбираем максимальную похожую (первую) картинку
        max_trashhold =  np.max(thresholds)
        max_id = np.where(thresholds == max_trashhold)[0][0]
        max_top_left =  top_left[max_id]
        max_bottom_right =  bottom_right[max_id]
        #print(part_id,max_id, max_trashhold,max_top_left,max_bottom_right)
        
        max_item = possible_old_items[max_id]        
        w, h = max_item.shape[::-1]
        res = cv.matchTemplate(max_item, temp, method) #threshlods  
        
        current_part_items = []
        if max_trashhold <= trashhold:
            current_part_items.append(None)
            parts_found.append(current_part_items)
            continue
        # смотрим сколько всего элементов нашлось в пределах 10%        
        min_trashhold = max_trashhold*trash_pers #trash_pers difference
        loc = np.where(res >= min_trashhold) 
        for i,pt in enumerate(zip(*loc[::-1])):
            current_part_items.append((pt, (pt[0] + w, pt[1] + h), part_id, max_trashhold))
            #print(part_id,'added item',current_part_items[-1])
            #cv.rectangle(temp, pt, (pt[0] + w, pt[1] + h), (0,0,0), 2)
        parts_found.append(current_part_items)
            
  
        #print(tabulate(items, headers=headers), '\n')
    #print (parts_found[11], sep='\n')
    
    #for i,part_id in enumerate(parts_found):
        #print(i, part_id)
    
    #todo внутри parts_found почистить пересечения внутри одного элемента, приоритет большему размеру
    # todo смотря на трешхолд и пересечение убрать 5-6 и 7-8 элементы
    
    parts_8=np.copy(parts_found[7])
    parts_7=np.copy(parts_found[6])
    
    #part 5 and 6
    if (parts_found[4][0] is not None and parts_found[5][0] is not None):
        if parts_found[4][0][3] > parts_found[5][0][3]:       
            to_clean =  parts_found[5] #part6 to remove 
            check_from = parts_found[5] #checking depends on part5
            check_from_copy = np.copy(parts_found[4])
            old_items = np.copy(parts_found[5]) # т.к. массив будет изменяться
        else:
            to_clean =  parts_found[4] #part5 to remove 
            check_from = parts_found[4] #checking depends on part6
            check_from_copy = np.copy(parts_found[5])
            old_items = np.copy(parts_found[4])# т.к. массив будет изменяться делаем копию проверяемых элементов        
            
            
        for check_item in check_from_copy:
            check_item = tuple(check_item)
            #print('checking for ', check_item)
            for item in old_items:
                item = tuple(item)
                #print('\tchecking ', item)
                if intersects((check_item[0], check_item[1]), (item[0],item[1])):
                    if (item in check_from):
                        check_from.remove(item)
                        #print('\t\tremoved ', item)  
            
            if len(to_clean) == 0:
                to_clean.append(None) 
    # part 5 and 6 end
    
    # part 7 and 8 
    if (parts_found[6][0] is not None and parts_found[7][0] is not None):
        if parts_found[6][0][3] > parts_found[7][0][3]:       
            to_clean =  parts_found[7] #part6 to remove 
            check_from = parts_found[7] #checking depends on part5
            check_from_copy = np.copy(parts_found[6])
            old_items = np.copy(parts_found[7]) # т.к. массив будет изменяться
        else:
            to_clean =  parts_found[6] #part5 to remove 
            check_from = parts_found[6] #checking depends on part6
            check_from_copy = np.copy(parts_found[7])
            old_items = np.copy(parts_found[6])# т.к. массив будет изменяться делаем копию проверяемых элементов        
        
        
        for check_item in check_from_copy:
            check_item = tuple(check_item)
            #print('checking for ', check_item)
            for item in old_items:
                item = tuple(item)
                #print('\tchecking ', item)
                if intersects((check_item[0], check_item[1]), (item[0],item[1])):
                    if (item in check_from):
                        check_from.remove(item)
                        #print('\t\tremoved ', item)  
            
            if len(to_clean) == 0:
                to_clean.append(None) 
        #part 7 and 8 end  
    
    # чистка дубликатов
    
    
    for item_parts in parts_found: # для всех 13 элементов
        for item1 in item_parts:
            for item2 in item_parts:
                if item1 is not None and item2 is not None and item1 != item2:
                    if intersects((item1[0],item1[1]),(item2[0],item2[1])):
                        item_parts.remove(item2)
    for item_parts in parts_found: # для всех 13 элементов
        for item1 in item_parts:
            for item2 in item_parts:
                if item1 is not None and item2 is not None and item1 != item2:
                    if intersects((item1[0],item1[1]),(item2[0],item2[1])):
                        item_parts.remove(item2)
        #print(item_parts)         
        
    # чистка end        
        
    
    parts_found_not_null = [x for x in parts_found if x != []]
    for i, found_parts in enumerate(parts_found_not_null):
        if found_parts == [None]:
            continue
            
        for single_part in found_parts:
            #print(single_part)
            if single_part is not None:
                cv.rectangle(temp, single_part[0], single_part[1], (0,0,0), 2)
    #show(temp)

            
    
             
    parts_found = [x for x in sum(parts_found, []) if x != None] # сжимаем в один массив все элементы и убираем None
    return parts_found

In [20]:
def change_colors(image):
    copy = (RGB(image)).copy()
    print(copy.height, copy.width)
    hsl_main_new = rgb_to_hsl(rgb_main_color_RGB) # пиксель темный
    for i in range(copy.width):  # for each column
        for j in range(copy.height):  # For each row  
            hsl_pixel = rgb_to_hsl(copy.getpixel((i,j)))
        
            if (hsl_pixel != (0,0,80) and hsl_pixel != (0,0,0) and hsl_pixel != (255,255,255) and hsl_pixel != (80,80,80)):    
                new_pixel = hsl_to_rgb((hsl_main_new[0], hsl_pixel[1], hsl_pixel[2]))
                #print(new_pixel)
                copy.putpixel((i,j),  new_pixel)
    return copy

In [15]:
def put_new_part(old_part, image):
    copy = image.copy()
    
    top_left = old_part[0]
    bottom_right = old_part[1]
    part_id = old_part[2]
    trash = old_part[3]
    #print(top_left, bottom_right, part_id, trash, sep = '\t')
    
    for i in range(top_left[0],bottom_right[0]):
        for j in range(top_left[1],bottom_right[1]):    
            pixel = copy.getpixel((i, j))
            copy.putpixel((i, j), white)
    
    x_size_zone = abs(bottom_right[0]-top_left[0])
    if x_size_zone % 2 == 0:
        x_size_zone = x_size_zone - 1        
    y_size_zone = abs(bottom_right[1] + 2 - top_left[1])
    if y_size_zone % 2 == 0:
        y_size_zone = y_size_zone - 1
    
    #x_size
    #y_size
    x_center = top_left[0]+math.floor(x_size_zone/2)+1
    y_center = top_left[1]+math.floor(y_size_zone/2)+1
    #print('Размер зоны для вставки: ',x_size_zone, y_size_zone)
    # максимальная высота/длина элемента    
    
    #получаем изображение которое вставляем нужного размера new_image
    new_image_path = new_parts_folder + '\\' + str(part_id) + '.eps'   
    new_image = Image.open(new_image_path)
    im_x_size, im_y_size = new_image.size
    
    scale_x = x_size_zone/im_x_size
    scale_y = y_size_zone/im_y_size
    
    #print('\tOLD esp size: ', new_image.size) #(100,100)
    
    scale = math.ceil(min(scale_x, scale_y))
    if scale > 1:    
        new_image.load(scale=scale) #high res увеличнение размера на максимальный
     
    #пересчитаем scale и уменьшим изображение
    im_x_size, im_y_size = new_image.size
    
    scale_x = x_size_zone/im_x_size
    scale_y = y_size_zone/im_y_size 
    scale = math.floor(min(scale_x, scale_y)*10000)/10000
    
    new_x = math.floor(im_x_size*scale)
    new_y = math.floor(im_y_size*scale)
    new_image = new_image.resize((new_x,new_y), Image.ANTIALIAS)
    
    #show(new_image.convert('RGB'))
    #print('\tNEW esp size: ', new_image.size) #(100,100)
    # изображение подготоволено new_image, вставляем его в центр x_center, y_center
    #print('center of new image: ',x_center, y_center)
    new_top_left = (x_center - math.ceil(new_x/2), y_center - math.ceil(new_y/2))
    
    copy.paste(new_image, new_top_left)
    
    return copy
#parts_found

In [41]:
#IM_PATH = sys.argv[1]
#IM_TO_PATH = sys.argv[2]

TRASHHOLD = 0.7
TRASH_PERS = 0.7
'''
if len(sys.argv) >= 3:
    if len(sys.argv) == 3:
        TRASHHOLD = float(sys.argv[2])
    
    if len(sys.argv) == 4:
        TRASHHOLD = float(sys.argv[2])
        TRASH_PERS = float(sys.argv[3])
'''
IM_PATH = 'examples/0.png'

cv_im = cv.imread(IM_PATH, 0)
parts_found = replace_old_parts(cv_im, trashhold=TRASHHOLD, trash_pers=TRASH_PERS) # получили список элементов которые нужно заменить   

PIL_im = Image.open(IM_PATH)
PIL_im = change_colors(PIL_im) 
PIL_im = black_to_white(PIL_im)
for part in parts_found:
    PIL_im = put_new_part(part, PIL_im)
    
#show(PIL_im)
PIL_im.save('output.png')
#rgb = RGB(PIL_im)
#show(rgb)
#new_color_image = change_colors(PIL_im)               
#show(new_color_image)

#print(PIL.EpsImagePlugin.has_ghostscript())



210 457
173.0 0.0 30.0
254.0 255.0 255.0
253.0 254.0 255.0
252.0 255.0 255.0
251.0 255.0 255.0
77.0 81.0 84.0
251.0 255.0 255.0
251.0 255.0 255.0
252.0 255.0 255.0
252.0 255.0 255.0
254.0 255.0 255.0
254.0 255.0 255.0
254.0 255.0 255.0
254.0 255.0 255.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
251.0 255.0 255.0
252.0 255.0 255.0
254.0 255.0 255.0
81.0 80.0 78.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 254.0 251.0
255.0 254.0 250.0
255.0 254.0

254.0 255.0 255.0
255.0 250.0 250.0
255.0 253.0 251.0
250.0 255.0 255.0
178.0 199.0 218.0
69.0 99.0 123.0
106.0 133.0 154.0
238.0 255.0 255.0
193.0 216.0 234.0
60.0 92.0 117.0
87.0 124.0 153.0
111.0 148.0 177.0
116.0 150.0 177.0
114.0 146.0 171.0
115.0 149.0 177.0
113.0 149.0 181.0
111.0 150.0 181.0
109.0 148.0 177.0
111.0 147.0 173.0
115.0 149.0 174.0
115.0 147.0 170.0
115.0 147.0 168.0
118.0 150.0 173.0
114.0 148.0 175.0
110.0 149.0 178.0
107.0 150.0 184.0
107.0 150.0 182.0
110.0 149.0 178.0
113.0 147.0 172.0
113.0 145.0 168.0
116.0 151.0 173.0
111.0 148.0 174.0
107.0 149.0 174.0
119.0 152.0 171.0
119.0 147.0 161.0
115.0 147.0 168.0
107.0 152.0 181.0
103.0 153.0 188.0
104.0 151.0 181.0
112.0 149.0 175.0
114.0 149.0 169.0
117.0 152.0 172.0
110.0 145.0 167.0
115.0 151.0 173.0
108.0 148.0 173.0
109.0 150.0 178.0
81.0 126.0 155.0
50.0 95.0 124.0
202.0 213.0 219.0
232.0 255.0 255.0
95.0 138.0 170.0
65.0 102.0 129.0
206.0 222.0 235.0
252.0 251.0 247.0
254.0 255.0 255.0
252.0 255.0 255.0
25

252.0 255.0 255.0
253.0 254.0 255.0
255.0 254.0 255.0
255.0 254.0 255.0
254.0 255.0 255.0
254.0 255.0 255.0
255.0 253.0 255.0
255.0 255.0 253.0
255.0 255.0 248.0
254.0 255.0 249.0
255.0 254.0 250.0
255.0 254.0 253.0
255.0 253.0 254.0
252.0 255.0 255.0
234.0 248.0 255.0
199.0 226.0 245.0
187.0 222.0 244.0
191.0 227.0 251.0
193.0 224.0 252.0
197.0 223.0 250.0
198.0 222.0 246.0
197.0 223.0 240.0
190.0 226.0 248.0
193.0 225.0 246.0
196.0 224.0 245.0
197.0 224.0 245.0
196.0 224.0 246.0
193.0 225.0 248.0
191.0 226.0 248.0
193.0 225.0 248.0
194.0 225.0 246.0
196.0 224.0 245.0
194.0 225.0 245.0
191.0 226.0 246.0
190.0 226.0 248.0
191.0 226.0 248.0
196.0 224.0 246.0
197.0 224.0 245.0
191.0 226.0 246.0
194.0 225.0 246.0
196.0 224.0 245.0
196.0 224.0 248.0
193.0 225.0 250.0
191.0 225.0 250.0
193.0 225.0 246.0
194.0 226.0 241.0
197.0 226.0 242.0
194.0 225.0 243.0
194.0 225.0 246.0
199.0 227.0 251.0
196.0 222.0 245.0
197.0 225.0 247.0
188.0 223.0 245.0
194.0 232.0 255.0
233.0 247.0 255.0
247.0 254.

237.0 255.0 255.0
240.0 255.0 255.0
238.0 255.0 255.0
236.0 255.0 255.0
234.0 255.0 255.0
231.0 255.0 255.0
230.0 255.0 255.0
233.0 255.0 255.0
241.0 255.0 255.0
247.0 255.0 255.0
247.0 255.0 255.0
244.0 255.0 255.0
241.0 255.0 255.0
242.0 255.0 255.0
244.0 255.0 255.0
238.0 255.0 255.0
144.0 179.0 209.0
40.0 88.0 128.0
183.0 179.0 176.0
238.0 255.0 255.0
113.0 157.0 186.0
58.0 99.0 127.0
203.0 223.0 234.0
255.0 255.0 253.0
255.0 254.0 251.0
252.0 255.0 255.0
254.0 255.0 255.0
254.0 255.0 255.0
251.0 255.0 255.0
249.0 255.0 255.0
251.0 255.0 255.0
255.0 254.0 253.0
251.0 255.0 255.0
255.0 254.0 250.0
255.0 253.0 250.0
249.0 255.0 255.0
171.0 197.0 214.0
66.0 103.0 122.0
101.0 137.0 153.0
234.0 255.0 255.0
190.0 216.0 231.0
63.0 94.0 115.0
84.0 121.0 148.0
107.0 151.0 176.0
108.0 153.0 176.0
108.0 153.0 176.0
107.0 151.0 178.0
108.0 151.0 183.0
105.0 152.0 182.0
109.0 151.0 175.0
113.0 149.0 171.0
112.0 149.0 176.0
107.0 150.0 185.0
106.0 150.0 187.0
105.0 151.0 184.0
106.0 151.0 182.0


61.0 102.0 130.0
61.0 102.0 130.0
61.0 102.0 130.0
61.0 102.0 130.0
61.0 102.0 130.0
61.0 102.0 130.0
61.0 102.0 130.0
61.0 102.0 130.0
61.0 102.0 130.0
61.0 102.0 130.0
61.0 102.0 130.0
61.0 102.0 130.0
61.0 102.0 130.0
61.0 102.0 130.0
61.0 102.0 130.0
61.0 102.0 130.0
61.0 102.0 130.0
61.0 102.0 130.0
61.0 102.0 130.0
62.0 101.0 130.0
62.0 101.0 130.0
62.0 101.0 130.0
61.0 102.0 130.0
61.0 102.0 130.0
61.0 102.0 130.0
61.0 102.0 130.0
61.0 102.0 130.0
61.0 102.0 130.0
61.0 102.0 130.0
61.0 102.0 130.0
61.0 102.0 130.0
61.0 102.0 130.0
61.0 102.0 130.0
61.0 102.0 130.0
61.0 102.0 130.0
59.0 103.0 130.0
70.0 106.0 128.0
141.0 169.0 183.0
237.0 255.0 255.0
220.0 246.0 255.0
87.0 118.0 136.0
69.0 101.0 122.0
192.0 227.0 249.0
178.0 213.0 235.0
71.0 103.0 124.0
85.0 116.0 136.0
221.0 247.0 255.0
236.0 255.0 255.0
139.0 167.0 181.0
72.0 108.0 130.0
57.0 101.0 128.0
61.0 103.0 127.0
59.0 103.0 128.0
58.0 104.0 128.0
57.0 105.0 128.0
58.0 104.0 127.0
59.0 104.0 125.0
61.0 103.0 125.0
59.0 1

...

189.0 212.0 228.0
98.0 132.0 159.0
54.0 97.0 131.0
58.0 106.0 146.0
59.0 111.0 150.0
54.0 103.0 136.0
59.0 100.0 128.0
140.0 167.0 186.0
240.0 253.0 255.0
254.0 252.0 253.0
255.0 252.0 248.0
255.0 254.0 251.0
255.0 255.0 253.0
255.0 254.0 255.0
255.0 254.0 255.0
254.0 255.0 255.0
254.0 255.0 255.0
254.0 255.0 255.0
254.0 255.0 255.0
254.0 255.0 255.0
255.0 254.0 255.0
254.0 255.0 253.0
252.0 255.0 253.0
247.0 255.0 255.0
152.0 178.0 193.0
49.0 86.0 112.0
152.0 188.0 214.0
235.0 255.0 255.0
132.0 151.0 165.0
72.0 95.0 113.0
193.0 225.0 248.0
194.0 226.0 249.0
72.0 95.0 113.0
132.0 151.0 165.0
235.0 255.0 255.0
152.0 188.0 214.0
49.0 86.0 112.0
152.0 178.0 193.0
247.0 255.0 255.0
255.0 255.0 253.0
254.0 255.0 255.0
254.0 255.0 255.0
255.0 255.0 253.0
255.0 255.0 253.0
254.0 255.0 255.0
255.0 255.0 253.0
255.0 255.0 253.0
252.0 255.0 255.0
245.0 255.0 255.0
155.0 177.0 191.0
43.0 88.0 121.0
156.0 187.0 208.0
240.0 255.0 255.0
129.0 152.0 168.0
66.0 97.0 118.0
199.0 222.0 240.0
249.0 255.0

132.0 151.0 165.0
237.0 255.0 255.0
153.0 187.0 212.0
49.0 86.0 112.0
151.0 178.0 195.0
244.0 255.0 255.0
255.0 254.0 255.0
255.0 254.0 255.0
255.0 254.0 253.0
255.0 254.0 255.0
255.0 255.0 253.0
255.0 254.0 253.0
255.0 254.0 255.0
255.0 254.0 255.0
255.0 255.0 253.0
252.0 255.0 255.0
245.0 255.0 255.0
155.0 177.0 190.0
43.0 88.0 121.0
156.0 187.0 208.0
240.0 255.0 255.0
129.0 152.0 168.0
66.0 97.0 118.0
199.0 222.0 240.0
249.0 255.0 255.0
255.0 254.0 251.0
255.0 254.0 251.0
254.0 255.0 255.0
254.0 255.0 255.0
255.0 254.0 255.0
254.0 255.0 255.0
255.0 254.0 251.0
252.0 255.0 255.0
255.0 254.0 248.0
255.0 253.0 246.0
248.0 255.0 255.0
198.0 225.0 244.0
66.0 97.0 118.0
129.0 152.0 166.0
235.0 255.0 255.0
151.0 189.0 212.0
46.0 88.0 113.0
150.0 179.0 195.0
247.0 255.0 255.0
251.0 255.0 252.0
251.0 255.0 255.0
250.0 255.0 254.0
254.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
255.0 255.0 253.0
25

123.0 145.0 159.0
121.0 144.0 162.0
124.0 147.0 165.0
122.0 141.0 156.0
129.0 148.0 162.0
121.0 144.0 160.0
118.0 150.0 173.0
82.0 121.0 150.0
53.0 98.0 131.0
185.0 216.0 237.0
240.0 255.0 255.0
109.0 132.0 148.0
70.0 101.0 122.0
172.0 195.0 213.0
249.0 255.0 255.0
255.0 254.0 251.0
255.0 254.0 251.0
254.0 255.0 255.0
254.0 255.0 255.0
255.0 254.0 255.0
254.0 255.0 255.0
255.0 254.0 251.0
254.0 255.0 255.0
255.0 254.0 255.0
255.0 253.0 250.0
254.0 255.0 255.0
190.0 209.0 224.0
65.0 100.0 128.0
82.0 119.0 146.0
223.0 242.0 255.0
254.0 255.0 255.0
156.0 160.0 159.0
80.0 102.0 116.0
60.0 101.0 129.0
59.0 104.0 135.0
64.0 101.0 127.0
68.0 99.0 119.0
70.0 101.0 121.0
71.0 99.0 120.0
71.0 99.0 121.0
69.0 100.0 121.0
69.0 99.0 123.0
69.0 99.0 123.0
69.0 99.0 123.0
69.0 99.0 123.0
71.0 99.0 121.0
72.0 99.0 120.0
72.0 99.0 120.0
72.0 99.0 120.0
72.0 99.0 120.0
71.0 99.0 121.0
69.0 100.0 121.0
68.0 100.0 123.0
68.0 100.0 123.0
68.0 101.0 118.0
68.0 100.0 121.0
68.0 100.0 123.0
69.0 100.0 121.0
6

255.0 254.0 253.0
255.0 254.0 255.0
255.0 253.0 253.0
255.0 254.0 250.0
255.0 255.0 250.0
255.0 255.0 253.0
254.0 255.0 255.0
254.0 255.0 255.0
254.0 255.0 255.0
255.0 255.0 253.0
255.0 255.0 250.0
255.0 255.0 251.0
255.0 254.0 255.0
255.0 255.0 251.0
255.0 254.0 255.0
251.0 254.0 255.0
250.0 255.0 255.0
254.0 255.0 255.0
249.0 254.0 255.0
119.0 140.0 157.0
63.0 102.0 131.0
63.0 104.0 134.0
132.0 160.0 181.0
209.0 229.0 240.0
226.0 248.0 255.0
220.0 248.0 255.0
217.0 249.0 255.0
220.0 249.0 255.0
227.0 250.0 255.0
223.0 245.0 255.0
224.0 250.0 255.0
221.0 247.0 255.0
224.0 251.0 255.0
222.0 249.0 255.0
222.0 250.0 255.0
220.0 248.0 255.0
221.0 248.0 255.0
222.0 249.0 255.0
224.0 250.0 255.0
222.0 248.0 255.0
224.0 250.0 255.0
222.0 248.0 255.0
226.0 252.0 255.0
220.0 247.0 255.0
219.0 248.0 255.0
220.0 249.0 255.0
222.0 249.0 255.0
221.0 250.0 255.0
219.0 248.0 255.0
222.0 251.0 255.0
221.0 250.0 255.0
220.0 247.0 255.0
225.0 251.0 255.0
221.0 247.0 255.0
223.0 249.0 255.0
221.0 248.0 

138.0 164.0 179.0
113.0 142.0 160.0
168.0 195.0 212.0
225.0 243.0 253.0
246.0 255.0 255.0
234.0 244.0 246.0
180.0 197.0 205.0
115.0 144.0 162.0
120.0 157.0 184.0
173.0 197.0 221.0
166.0 192.0 215.0
136.0 162.0 185.0
116.0 138.0 159.0
176.0 192.0 207.0
232.0 242.0 251.0
247.0 255.0 255.0
232.0 246.0 255.0
177.0 200.0 218.0
115.0 143.0 165.0
124.0 155.0 176.0
169.0 198.0 216.0
168.0 195.0 214.0
130.0 162.0 183.0
142.0 182.0 208.0
43.0 88.0 121.0
169.0 184.0 189.0
230.0 255.0 255.0
112.0 156.0 191.0
62.0 97.0 127.0
207.0 221.0 234.0
255.0 254.0 255.0
255.0 255.0 251.0
251.0 255.0 255.0
248.0 255.0 255.0
251.0 255.0 255.0
255.0 254.0 255.0
255.0 254.0 255.0
255.0 255.0 253.0
255.0 255.0 250.0
252.0 255.0 255.0
255.0 254.0 248.0
255.0 254.0 246.0
247.0 255.0 255.0
195.0 226.0 247.0
65.0 97.0 118.0
130.0 152.0 163.0
240.0 255.0 255.0
156.0 187.0 208.0
49.0 85.0 111.0
149.0 176.0 195.0
234.0 248.0 255.0
196.0 209.0 217.0
135.0 151.0 164.0
125.0 139.0 150.0
184.0 195.0 199.0
224.0 228.0 231.0
